In [1]:
import pandas as pd 
import numpy as np

subset_method = "Union"
correlation_type = "Pearson"
organ1 = "*"
organ2 = "average"

correlations_raw = pd.read_csv(f"../../data/page6_LinearXWASCorrelations/correlations/Correlations_{subset_method}_{correlation_type}.csv", usecols=["env_dataset", "organ_1", "organ_2", "corr", "sample_size"]).replace("\\*", "*")

correlations = correlations_raw[(correlations_raw.organ_1 == organ) & (correlations_raw.organ_2 != organ)].fillna(0)
sample_sizes_2d = pd.pivot_table(correlations, values="sample_size", index=["env_dataset"], columns=["organ_2"])
correlations_2d = pd.pivot_table(correlations, values="corr", index=["env_dataset"], columns=["organ_2"])


In [2]:
labels_x = correlations_2d.T.copy().columns
labels_y = correlations_2d.T.copy().index
heat_correlations = correlations_2d.T.copy().values
heat_sample_sizes = sample_sizes_2d.T.copy().values

In [3]:
def f(x):
    if x <= 0:
        x1 = 255 * (x + 1)
        x1 = round(x1, 5)
        return "rgba(%s, %s, %s, 0.85)" % (255, int(x1), int(x1))
    else:
        x2 = 255 * (1 - x)
        x2 = round(x2, 5)
        return "rgba(%s, %s, %s, 0.85)" % (int(x2), int(x2), 255)


def get_colorscale(df):
    min = df.min().min()
    max = df.max().max()
    abs = np.abs(min / (min - max))
    if abs > 1:
        colorscale = [[0, f(min)], [1, f(max)]]
    else:
        colorscale = [[0, f(min)], [abs, "rgba(255, 255, 255, 0.85)"], [1, f(max)]]
    return colorscale

In [4]:
from plotly.graph_objs import Heatmap, Figure

colorscale = get_colorscale(heat_correlations)

hovertemplate = "Correlation : %{z} <br>Organ x : %{x} <br>Organ y : %{y} <br>Sample Size : %{customdata}"

heatmap = Heatmap(
    x=labels_x,
    y=labels_y,
    z=heat_correlations,
    colorscale=colorscale,
    customdata=heat_sample_sizes,
    hovertemplate=hovertemplate,
)

Figure(heatmap)

In [20]:
heat_correlations = correlations.loc[labels, labels].values
colorscale = get_colorscale(heat_correlations)

heat_sample_sizes = sample_sizes.loc[labels, labels].values
hovertemplate = "Correlation : %{z} <br>Organ x : %{x} <br>Organ y : %{y} <br>Sample Size : %{customdata}"

In [23]:
heatmap = Heatmap(
    x=labels,
    y=labels,
    z=heat_correlations,
    colorscale=colorscale,
    customdata=heat_sample_sizes,
    hovertemplate=hovertemplate,
)

heatmap["x"] = fig["layout"]["xaxis"]["tickvals"]
heatmap["y"] = fig["layout"]["xaxis"]["tickvals"]

fig.update_layout(
    yaxis={
        "domain": [0, 0.85],
        "mirror": False,
        "showgrid": False,
        "zeroline": False,
        "ticktext": labels,
        "tickvals": fig["layout"]["xaxis"]["tickvals"],
        "showticklabels": True,
        "ticks": "outside",
    }
)

fig.add_trace(heatmap)

fig["layout"]["width"] = 1100
fig["layout"]["height"] = 1100

fig.show()